In [ ]:
model_file = '/content/drive/MyDrive/llm/llama-2-7b-chat.ggmlv3.q8_0.bin'

In [ ]:
!pip install langchain

In [ ]:
!pip install sentence-transformers

In [ ]:

!pip install langchain_community

In [ ]:
!pip install ctransformers

In [ ]:
!pip install faiss-gpu

In [ ]:
pip install pypdf

In [ ]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings

from langchain.vectorstores import FAISS

import box
from langchain.llms import CTransformers

In [ ]:
dir_files = DirectoryLoader("/content/drive/MyDrive/llm", glob= "*.pdf", loader_cls= PyPDFLoader)

In [ ]:
docs = dir_files.load()

In [ ]:
splited_text = RecursiveCharacterTextSplitter(chunk_size = 550, chunk_overlap = 50)

In [ ]:
documents_splitted = splited_text.split_documents(docs)

In [ ]:
documents_splitted

[Document(metadata={'source': '/content/drive/MyDrive/llm/bank_statement.pdf', 'page': 0}, page_content='Account Number:\n1234567890\nAccount Holder:\nJohn Doe\nBranch Name:\nMain Branch\nStatement Date:\n15-Nov-2023\nCIF No:\n89725346219\nIFSC Code:\nSBIN000234\nBanalce As of:\n156780'),
 Document(metadata={'source': '/content/drive/MyDrive/llm/bank_statement.pdf', 'page': 1}, page_content='Date\nDescription\nTrans. ID\nAmount\nType\nBalance\n2022-11-09\nWhether mouth use trip.\nAIE1HQ\n156780.0\nCredit\n156780.0\n2022-11-10\nFull many kind.\nRCCMS4\n16.62\nCredit\n156796.62\n2022-11-10\nOur produce minute.\nIR6B4I\n120.29\nDebit\n156676.33\n2022-11-11\nEither all wrong improve.\nXWKYLO\n465.96\nCredit\n157142.29\n2022-11-12\nHe store fly success few.\nG7IP4A\n312.42\nCredit\n157454.71\n2022-11-12\nGet notice figure big the\n68A4KU\n351.72\nDebit\n157102.99\n2022-11-14\nActivity this network.\n6GNDNJ\n63.59\nDebit\n157039.4\n2022-11-14\nGeneration by war he give\nD7RAS7\n142.11'),
 Do

In [ ]:
embedding = HuggingFaceEmbeddings(model_name = "sentence-transformers/all-MiniLM-L6-v2",
                                  model_kwargs = {"device": "cpu"})

<ipython-input-14-619b9b04e661>:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = HuggingFaceEmbeddings(model_name = "sentence-transformers/all-MiniLM-L6-v2",
/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a tok

In [ ]:
vect = FAISS.from_documents(documents_splitted, embedding)

In [ ]:
vect.save_local("db/faiss")

In [ ]:
llm_model = CTransformers(
    model = model_file,
    model_type = "llama",
    config = {"max_new_tokens":500, "temperature": 0.01}
)

In [ ]:
# A low temperature (close to 0) makes the output more deterministic (less creative).
# A high temperature (e.g., 1.0) increases randomness, leading to more diverse outputs.

In [ ]:
# prompt engineering

it is the process of designing and refining prompts effectively interact with language models(like gpt,LLama etc)

How you ask : A well designed prompt can drastically improve response qulity
what you include :provide clear instructions and relevent context for better o/p



In [ ]:
"""
you are a medical chatbot /
you should respond with some medications or tablets /

if you do not know the answer, do not makeover, Just respond as 'No Idea' /

Example: what is that tablet name?
        bot answer : <tablet name>

"""

In [ ]:
prompt = """

use the following information and answer the question based on that /
provide only useful informations /
if you do not know, do not try to make wrong answers /
if you do not know, just respond as 'I do not know' /

show only the answer alone /
Example: what is the name in doc?
       output: Raj

Context: {context}
Questions: {question}


"""

In [ ]:
# making prompt template

from langchain import PromptTemplate
from langchain.chains import RetrievalQA

In [ ]:
# creating sub functions

def sqa_prompt():
  prompt_temp = PromptTemplate(template = prompt, input_variables = ["context", "question"])
  return prompt_temp

In [ ]:
def ret_answer(llm_model_name,prompt_name,v_db):
  ques_ans = RetrievalQA.from_chain_type(
      llm = llm_model_name,
      chain_type = "stuff",
      chain_type_kwargs = {"prompt": prompt_name},
      retriever = v_db.as_retriever(search_kwargs={'k':2})
  )
  return ques_ans

In [ ]:
#"stuff": Concatenates all retrieved documents into a single input for the language model.

In [ ]:
def setup_mode():
  embb = HuggingFaceEmbeddings(model_name = "sentence-transformers/all-MiniLM-L6-v2",
                                  model_kwargs = {"device": "cpu"})
  vdb = FAISS.load_local("/content/db/faiss", embb, allow_dangerous_deserialization= True)

  pmt = sqa_prompt()

  get_answer = ret_answer(llm_model, pmt, vdb)
  return get_answer

In [ ]:
# final load run of llm model

llm_qa_model = setup_mode()

In [ ]:
asked_question = 'What is account number?'

In [ ]:
llm_qa_model({'query':asked_question})

<ipython-input-28-9e25680c5c04>:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  llm_qa_model({'query':asked_question})


{'query': 'What is account number?', 'result': 'Answer: 1234567890'}